# Kvantni algoritmi 4

## Kvantna kola za računanje klasičnih funkcija

Pre nego što uvedemo Šorov algoritam, potrebno je napisati kodove za kvantno sabiranje i množenje po modulu celog broja. Objasnićemo ukratko kako to radi i implementirati kod. Algoritmi za ove operacije su opisani u poglavlju $6$ ove [knjige](https://mitpress.mit.edu/9780262526678/quantum-computing/).

#### Glavna ideja:

Kvantna kola bez kvantnog merenja su *reverzibilna*, jer je unitarna transformacija invertibilna. Klasična kola 

Reverzibilno klasično kolo koje ima $n$ bitova na ulazu i $n$ bitova na izlazu se može predstaviti kao bijekcija $f: \mathbb{Z}^n \xrightarrow{} \mathbb{Z}^n$, odnosno kao permutacija skupa $\mathbb{Z}^n$. Kada nam je ova funkcija poznata, kvantno kolo koje izračunava $f$ se može, na baznim vektorima $\ket{x}$, odrediti jednačinom:
\begin{align*}
\sqcup_f \ket{x} = \sqcup_f \ket{f(x)}.
\end{align*}

U slučaju da je $f: \mathbb{Z}^n \xrightarrow{} \mathbb{Z}^m$, za $n \neq m$, ovo klasično kolo možemo prevesti u reverzibilno klasično kolo definisanjem funkcije $g: \mathbb{Z}^{n + m} \xrightarrow{} \mathbb{Z}^{n + m}$:
\begin{align*}
g(x, y) = (x, y \oplus f(x)),
\end{align*} za sve $x \in \mathbb{Z}^n$ i $y \in \mathbb{Z}^m$, gde je $\oplus$ bitovski $XOR$.

Odavde sledi konstrukcija za kvantno kolo $\sqcup_f$ definisano jednačinama na baznim vektorima $\ket{x} \otimes \ket{y}$:
\begin{align*}
\sqcup_f \ket{x} \ket{y} = \ket{x} \ket{y \oplus f(x)}.
\end{align*}

### NOT:

Kvantno kolo $X$ je analog klasičnom kolu $NOT$. Zaista, bijekcija koja računa $NOT$ je $f(0) = 1$, $f(1) = 0$. Odavde je $\sqcup_f = X$.

### XOR:

Standardna verzija funkcije koja izračunava $XOR$ je $f(x, y) = x \oplus y$. Broj bitova na ulazu je $2$, a broj bitova na izlazu je $1$. Ako bismo pratili postupak koji smo dali kada je $n \neq m$, reverzibilno klasično kolo bi imalo $3$ bitova na ulazu i na izlazu. U ovom slučaju može i efikasnije, ako se uzme $f(x, y) = (x, x \oplus y)$, što daje $2$ bitova na ulazu i na izlazu. Dakle, $\sqcup_f$ je određeno jednačinama:
\begin{align*}
\sqcup_f \ket{x} \ket{y} = \ket{x} \ket{y \oplus x},
\end{align*} za sve bazne vektore $\ket{x}\ket{y}$. Odavde nije teško dokazati da je $\sqcup_f = CNOT$.

### AND:

Standardna verzija funkcije koja izračunava $AND$ je $f(x, y) = 1$ ako i samo ako je $x = 1$ i $y = 1$. Reverzibilna verzija se može konstruisati postupkom koji smo dali iznad, čime dobijamo $g(x, y, z) = (x, y, z \oplus f(x, y))$. Odavde se lako dobija $\sqcup_f$. Primetimo da će vrednost $f(x, y)$ biti $1$ ako i samo ako su $x$ i $y$ jednaki $1$. U tom slučaju se vrednost od $z$ invertuje, a inače vrednost trećeg bita na izlazu ostaje ista. Na kvantnom računaru ovo ponašanje na baznim vektorima ima $CCNOT$. Dakle, $\sqcup_f = CCNOT$.

### NAND:

$NAND$ predstavlja kompoziciju $AND$ i $NOT$. Nakon dobijanja $\ket{x}\ket{y}\ket{z \otimes f(x, y)}$, primenom $X$ na trećem kubitu se dobija $\ket{x}\ket{y}\ket{z \otimes NOT(f(x, y))} = \ket{x}\ket{y}\ket{z \otimes NAND(x, y)}$. Kvantno kolo koje implementira ovo je $(I \otimes I \otimes X)CCNOT$.

### Alternativna konstrukcija:

Moguće je sve četiri prethodne operacije objediniti jednom operacijom $CCNOT$, ako se argumenti dobro nameste. To je zahvaljujući tome što važe naredne jednakosti:
\begin{align*}
CCNOT \ket{11}\ket{x} &= \ket{11}\ket{NOT(x)}, \\
CCNOT \ket{1}\ket{x}\ket{y} &= \ket{1}\ket{x}\ket{x \oplus y}, \\
CCNOT \ket{x}\ket{y}\ket{0} &= \ket{x}\ket{y}\ket{AND(x, y)}, \\
CCNOT \ket{x}\ket{y}\ket{1} &= \ket{x}\ket{y}\ket{NAND(x, y)},
\end{align*} za bazne vektore $\ket{x}$ i $\ket{y}$.

### Sabirač:

#### Sabiranje dva bita:

Ova konstrukcija je elementarna i biće korišćena da definiše složenije sabirače. Na ulazu se nalaze bitovi $a$ i $b$ koje treba sabrati i prenosni bit $c$. Dakle, rezultat treba da bude $(a +_2 b +_2 c)$. Na klasičnom računaru, ovo ponašanje definiše funkcija $f(c, a, b) = (a \oplus b \oplus c)$. Da bi ta funkcija postala reverzibilna, možemo definisati $g(c, a, b) = (c, a, c \oplus a \oplus b)$. Pomoću $CNOT$ kvantnih kola, možemo na trećem kubitu dodavati $XOR$ operacije, kao što vidimo u kodu ispod.

##### Sabiranje dva bita u Q#

In [1]:
import qsharp

In [2]:
%%qsharp

open Microsoft.Quantum.Diagnostics;

operation BitAdd(c : Qubit, a : Qubit, b : Qubit) : Unit is Adj + Ctl {
    CNOT(a, b);
    CNOT(c, b);
}

##### Računanje prenosa:

Prilikom sabiranja bitova $a$, $b$ i $c$, do prenosa na sledeću poziciju dolazi ako su bar dva bita vrednosti $1$. Izraz koji računa da li je došlo do prenosa je $(a \land b) \oplus (c \land (a \oplus b))$. Ovo se sada može lako implementirati kao kvantno kolo sa četiri kubita.

##### Računanje prenosa u Q#:

In [3]:
%%qsharp

operation BitCarry(c : Qubit, a : Qubit, b : Qubit, r : Qubit) : Unit is Adj + Ctl {
    // smeštamo AND(a, b) u r
    CCNOT(a, b, r);
    // smeštamo XOR(a, b) u b
    CNOT(a, b);
    // u r je AND(a, b)
    // ako je AND(c, XOR(a, b)), onda treba r da invertujemo
    // inače ne treba ništa da radimo
    // trenutno je u b XOR(a, b)
    CCNOT(c, b, r);
    // u r je sada XOR((XOR(a, b) AND c), AND(a, b))
    // vraćamo b na staru vrednost
    CNOT(a, b);
    // na kraju je izlaz |a>|b>|c>|XOR((XOR(a, b) AND c), AND(a, b))>
}

#### Višebitni sabirač:

Pokazaćemo kako se može sabiranje dva $n$-bitna broja izvesti na kvantnom računaru. Kvantno kolo koje to izvodi treba da zadovoljava sledeću jednakost:
\begin{align*}
Add \ket{c}\ket{a}\ket{b} = \ket{c}\ket{a}\ket{(a +_{2N} b +_{2N} c)},
\end{align*} za sve bazne vektore $\ket{a}\ket{b}\ket{c}$, gde je $N = 2^{n}$. Registar $b$ ima $n + 1$ bit, dok $a$ i $b$ sadrže po $n$ bitova. Početno stanje od $c$ je $\ket{0 \ldots 0}$, a $\ket{b} = \ket{0}\ket{b'}$.

Bez daljih objašnjenja, implementiraćemo kod iz knjige.

##### Višebitni sabirač u Q#:

In [4]:
%%qsharp

open Std.Arrays;

operation Add(c : Qubit[], a : Qubit[], b : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(c);
    if n == 1 {
        BitCarry(c[0], a[0], b[1], b[0]);
        BitAdd(c[0], a[0], b[1]);
    } else {
        BitCarry(c[n - 1], a[n - 1], b[n], c[n - 2]);
        Add(Most(c), Most(a), Most(b));
        Adjoint BitCarry(c[n - 1], a[n - 1], b[n], c[n - 2]);
        BitAdd(c[n - 1], a[n - 1], b[n]);
    }
}

use c = Qubit[5];
use a = Qubit[5];
use b = Qubit[6];

X(a[2]); X(a[1]); // a --> 01100 = 12
X(b[5]); X(b[2]); // b --> 001001 = 9

Add(c, a, b);
// c i a treba da ostanu isti
// b treba da sadrži 21
// 00000 01100 010101

DumpMachine();

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|0000001100010101⟩</span>
  </td>
  <td>
    <span>1.0000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="100"></progress>
    <span>100.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = |0000001100010101\rangle$

#### Bibliotečka funkcija za računanje zbira u Q#:

U biblioteci **Std.Arithmetic**, nalazi se funkcija koja računa zbir dva broja na kvantnom računaru. Potpis te funkcije je **operation AddLE(xs : Qubit[], ys : Qubit[], zs : Qubit[]) : Unit is Adj**. Zvanična [dokumentacija](https://learn.microsoft.com/en-us/qsharp/api/qsharp-lang/std.arithmetic/addle) kaže da registri **xs**, **ys** i **zs** smeštaju najznačajniji bit na poslednje mesto. Takođe, **xs** i **ys** su iste dužine, a dužina od **zs** je barem onolika kolika je od **xs**. Pre izvršavanja, **zs** bi trebalo da bude u stanju $\ket{0 \ldots 0}$. Nakon izvršavanja **AddLE**, u **zs** će biti zbir **xs** i **ys**.

Osim ove, postoji u istoj biblioteci i **operation RippleCarryCGAddLE(xs : Qubit[], ys : Qubit[], zs : Qubit[]) : Unit is Adj**. Obe funkcije rade isto, samo što **RippleCarryCGAddLE** radi efikasnije od **AddLE**.

In [5]:
%%qsharp

open Std.Arithmetic;

for q in b {
    Reset(q);
}

for q in a {
    Reset(q);
}

for q in c {
    Reset(q);
}

// smestićemo a = 5
// c = 7
// b = 0

X(a[0]); X(a[2]);
X(c[0]); X(c[1]); X(c[2]);

AddLE(c, a, b);

// u b treba da bude 12, odnosno 001100
// krajnje stanje je (c, a, b) = (11100, 10100, 001100)

DumpMachine();

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|1110010100001100⟩</span>
  </td>
  <td>
    <span>1.0000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="100"></progress>
    <span>100.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = |1110010100001100\rangle$

### Množenje na kvantnom računaru:

Ne postoji bibliotečka funkcija za računanje proizvoda u Q#. Ovde ćemo prikazati kako se može raditi množenje po modulu $2^n$, za neko $n$. Algoritam koji sada prikazujemo je predložila grupa autora u sledećem [radu](https://ieeexplore.ieee.org/document/9262868).

Na ulazu u kvantno kolo su nam dati $\ket{a}$, $\ket{b}$, $\ket{c_0}$, $\ket{c_1}$. Registri $a$, $b$, $c_0$ i $c_1$ su veličine $n$. Rezultat množenja je $a \cdot_N b$, gde je $N = 2^n$, i rezultat se smešta u registar $c_1$. Početno stanje registara $c_0$ i $c_1$ je $\ket{0 \ldots 0}$. Takođe, pošto je $\ket{c_0}$ u stanju $\ket{0 \ldots 0}$ pre i posle izvršavanja operacije, možemo da pretvorimo $c_0$ u pomoćne kubite.

In [6]:
%%qsharp

open Std.Arrays;

operation Multiply(a : Qubit[], b : Qubit[], c1 : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);
    use c0 = Qubit[n];
          
    // u registar c1 se smešta prvi parcijalni proizvod, odnosno a[0] * b
    for i in 0..n-1 {
        CCNOT(a[0], b[i], c1[i]);
    }

    for i in 1..n-1 {
        within {
            // u c0 smeštamo naredni parcijalni proizvod bez bitova posle n-tog
            for j in 0..n-1-i {
                CCNOT(a[i], b[j], c0[i + j]);
            }
        } apply {
            // ova funkcija efikasno dodaje vrednost od c0 na c1
            RippleCarryCGIncByLE(c0, c1);
        }
    }
}

// testirajmo na a, c
// trenutno je a = 5, c = 7
// definišemo c1
use c1 = Qubit[5];

Multiply(a, c, c1);

// a * c = 35 i pošto je registar 5 bitova, onda mod 2^5 = 32
// dakle, očekujemo 3 na izlazu c1
// 00011 je zapis za 3, ali pošto gledamo obrnuto, onda je 11000

DumpMachine();

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|111001010000110011000⟩</span>
  </td>
  <td>
    <span>1.0000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="100"></progress>
    <span>100.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = |111001010000110011000\rangle$

Ako pogledamo poslednjih pet kubita, vidimo da se nalaze u stanju $\ket{11000}$, što je upravo ono što smo očekivali. Ovde smo prvi put primenili blokove **within** i **apply**. Dosta kvantnih algoritama se zasniva na izračunavanju oblika $V U V^{\dagger}$. Zbog toga je jedna od naredbi u Q# within-apply. U bloku **within** pišemo šta radi operacija $V$, a u bloku apply pišemo šta radimo operacija $U$. Pošto će operacija $V$ biti invertovana, bitno je da se u njoj primenjuju samo **Adj** operacije.

### Kvantno stepenovanje u Q#:

Argumenti ove operacije su registri $\ket{a}$, $\ket{x}$ i $\ket{r}$. Registar $t$ će biti pomoćni registar. Rezultat smeštamo u registar $\ket{r}$. Ako je $r$ registar dužine $n$, onda ovo realizuje stepenovanje po modulu $2^n$. Zbog jednostavnosti, smatraćemo da su dužine registra $a$ i $x$ takođe $n$. Registri $\ket{t}$ i $\ket{r}$ su jednake dužine.

Optimalni algoritam, koji radi sa $O(\log n)$ kvantnih kola nije trivijalan za implementaciju. Zato ćemo pokazati jednostavnije rešenje, koje radi sa $O(n)$ kvantnih kola. Osnovna ideja je da oduzimamo $1$ od $x$ dok n bude $x = 0$.

In [7]:
%%qsharp

// kvadrira dva a i smešta rezultat u r
operation Square(a : Qubit[], r : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(r);
    use r0 = Qubit[n];
    Multiply(a, a, r);
}

operation Copy(a : Qubit[], b : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);
    for i in 0..n-1 {
        CNOT(a[i], b[i]);
    }
}